In [1]:
!!pip install mcap

['Requirement already satisfied: mcap in ./venv/lib/python3.10/site-packages (1.1.1)',
 'Requirement already satisfied: zstandard in ./venv/lib/python3.10/site-packages (from mcap) (0.22.0)',
 'Requirement already satisfied: lz4 in ./venv/lib/python3.10/site-packages (from mcap) (4.3.3)']

In [4]:
import json
import sys
from time import time_ns,sleep

from mcap.writer import Writer,CompressionType

ONE_HOUR_IN_NANOSECONDS = 3600000000000

compression_types = {
    "LZ4": CompressionType.LZ4,
    "NONE": CompressionType.NONE,
    "ZSTD": CompressionType.ZSTD,

}

for compression_name, _compression_type in compression_types.items():

    with open(f"basic_{compression_name}.mcap", "wb") as stream:
        writer = Writer(stream,compression=_compression_type)
    
        writer.start()
    
        schema_id = writer.register_schema(
            name="sample",
            encoding="jsonschema",
            data=json.dumps(
                {
                    "type": "object",
                    "properties": {
                        "value": {
                            "type": "integer",
                        }
                    },
                }
            ).encode('utf-8'),
        )
    
        channel_id = writer.register_channel(
            schema_id=schema_id,
            topic="integer_topic",
            message_encoding="json",
        )
    
    
        for i in range(10):
            sleep(1)
            writer.add_message(
                channel_id=channel_id,
                log_time=ONE_HOUR_IN_NANOSECONDS * i,
                data=json.dumps({"value": i},ensure_ascii=False).encode('utf-8'),
                publish_time=ONE_HOUR_IN_NANOSECONDS * i,
                sequence=i
            )
    
        writer.finish()